In [ ]:
# | hide
# | default_exp func
import nbdev
from nbdev.showdoc import *

nbdev.nbdev_export()

# Functions that operate on tensors


In [ ]:
# | exporti

import math
import numpy as np

import tidygrad.ops as ops
from tidygrad import Tensor

In [ ]:
#| export
def add(a: Tensor, b: Tensor, name=None) -> Tensor:
    """Add two tensors"""
    return ops.Add(a, b, name=name).out

def sub(a: Tensor, b: Tensor, name=None) -> Tensor:
    """Subtract two tensors"""
    return ops.Sub(a, b, name=name).out

def mul(a: Tensor, b: Tensor, name=None) -> Tensor:
    """Multiply two tensors"""
    return ops.Mul(a, b, name=name).out

def div(a: Tensor, b: Tensor, name=None) -> Tensor:
    """Divide two tensors (a/b)"""
    return ops.Div(a, b, name=name).out

def neg(a: Tensor, name=None) -> Tensor:
    """Negate a tensor (-a)"""
    return ops.Neg(a, name=name).out

def pow(a: Tensor, power: Tensor, name=None) -> Tensor:
    """Raise a tensor to a power (a**power)"""
    return ops.Pow(a, power, name=name).out

def log(a: Tensor, name=None) -> Tensor:
    """Take the natural logarithm of a tensor"""
    return ops.Log(a, name=name).out

def exp(a: Tensor, name=None) -> Tensor:
    """Exponentiate a tensor"""
    return ops.Exp(a, name=name).out

def logexp(a: Tensor, name=None) -> Tensor:
    """Exponentiate a tensor"""
    return ops.ExpLog(a, name=name).out

def matmul(a: Tensor, b: Tensor, name=None) -> Tensor:
    """Matrix multiplication of two tensors"""
    return ops.Matmul(a, b, name=name).out

def sum(a: Tensor, name=None, axis=None, keepdims=False) -> Tensor:
    """Sum-reduce a tensor along the given axis (int or tuple of ints)"""
    return ops.Sum(a, name=name, axis=axis, keepdims=keepdims).out

def broadcast(a: Tensor, target_shape, name=None) -> Tensor:
    """Broadcast a tensor to the given shape"""
    return ops.Broadcast(a, target_shape, name=name).out

def slice(a: Tensor, key, name=None) -> Tensor:
    return ops.Slice(a, key, name=name).out

def transpose(a: Tensor, dim0, dim1, name=None) -> Tensor:
    """Transpose a tensor"""
    return ops.Transpose(a, dim0, dim1, name=name).out

def dropout(x: Tensor, p=0.5, training=True) -> Tensor:
    """Apply Dropout to a tensor"""
    return ops.Dropout(x, p_drop=p, training=training).out

def embedding(input: Tensor, indices, name=None) -> Tensor:
    return ops.Embedding(input, indices, name=name).out

In [ ]:
# | export
def sigmoid(input: Tensor, name=None) -> Tensor:
    return ops.Sigmoid(input, name=name).out

def tanh(input: Tensor, name=None) -> Tensor:
    return 2 * sigmoid(2 * input, name=name) - 1

def relu(input: Tensor, name=None) -> Tensor:
    return ops.Relu(input, name=name).out

In [ ]:
#| export
def sigmoid_gelu(x: Tensor):
    return x * sigmoid(1.702 * x)

def gelu(input: Tensor):
    return (0.5 * input * (1.0 + tanh(math.sqrt(2.0 / math.pi) * (input + 0.044715 * input.pow(3)))))

In [ ]:
#| export
def softmax(input: Tensor, name=None) -> Tensor:
    exp = input.exp()
    return exp.div(exp.sum(axis=-1, keepdims=True), name=name)

In [ ]:
#| export

def layer_norm(x: Tensor, w: Tensor, b: Tensor, eps=1e-5) -> Tensor:
    mu = x.mean(axis=-1, keepdims=True)
    sigma = x.std(axis=-1, keepdims=True, correction=0)
    if sigma.data.any() == 0:
        print("x", x)
        print("w", w)
        print("b", b)
        print("mu", mu)
        print("sigma", sigma)
        raise ValueError("sigma is zero")

    return ((x-mu) / (sigma+eps)) * w + b  #  tensor[10, 768] n=7680 (30Kb) x∈[-0.788, 0.579] μ=-0.005 σ=0.106

In [ ]:
#| export

def stack(tensors: list[Tensor], axis=0, name=None):
    return ops.Stack(tensors, axis=axis, name=name).out

def concat(tensors: list[Tensor], axis=0, name=None):
    return ops.Concat(tensors, axis=axis, name=name).out


In [ ]:
# | export
def BCE_loss(logits: Tensor, target: Tensor, reduction="mean"):
    loss = logits - logits*target + ops.ExpLog(-logits).out
    if reduction == "mean":
        return loss.mean()
    if reduction == "sum":
        return loss.sum()
    assert 0, "Invalid reduction"

In [ ]:
# | export

def CrossEntropy_loss(logits: Tensor, target: Tensor, reduction="mean"):
    if not isinstance(target, Tensor):
        target = Tensor(target)
    sm = softmax(logits)
    loss = -target * sm.log()

    if reduction == "mean":
        return loss.mean(axis=-1, keepdims=True)
    if reduction == "sum":
        return loss.sum(axis=-1, keepdims=True)
    assert 0, "Invalid reduction"